# Trust Pilot Webscraping
Author: Stefanie Arlt

In [2]:
# import
import urllib.request
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from bs4 import UnicodeDammit

### Energy distributors operating in Germany

37 distributors in Germany are listed on two pages on the Trustpilot web site for the category of 'power supply company'.
 - Page 1 with 20 entries.
 - Page 2 with 17 entries.

The standard sorting of the suppliers is "according to relevance". This filter shows all energy suppliers sorted by highest TrustScore and numbers of votes. In addition, the companies have to fulfill the following conditions, to ensure that the companies with the highest votes receive up-to-date customer feedback:
 - The supplier needs to have received at least 25 evaluations in the last 12 months.
 - The supplier must have the status "asking for evaluation".

We would like to extract the name, score, number of votes and business location of each energy distributor.

In [4]:
# Store the url of the first page
url1_tp = "https://de.trustpilot.com/categories/electric_utility_company"
#Retrieve the HTML code of the page
page1 = urllib.request.urlopen(url1_tp)

In [5]:
# Store the url of the second page
url2_tp = "https://de.trustpilot.com/categories/electric_utility_company?page=2"
#Retrieve the HTML code of the page
page2 = urllib.request.urlopen(url2_tp)

In [6]:
# set HTML code as HTML file with parser
page1_soup = bs(page1, "html.parser")
page2_soup = bs(page2, "html.parser")

In [7]:
# first header on page 1
print(page1_soup.h1.text)

Spitzenreiter in der Kategorie Stromversorgungsunternehmen


In [8]:
# title of page 1
print(page1_soup.title.text)

Die besten Unternehmen in der Kategorie Stromversorgungsunternehmen auf Trustpilot


In [9]:
#number of children in page1_soup
len(list(page1_soup.descendants))

6087

In [10]:
#Display lines with list of energy suppliers
page1_soup.prettify().splitlines()[1120:2000]

['             </option>',
 '             <option value="BY">',
 '              Weißrussland',
 '             </option>',
 '             <option value="EH">',
 '              Westsahara',
 '             </option>',
 '             <option value="CF">',
 '              Zentralafrikanische Republik',
 '             </option>',
 '             <option value="CY">',
 '              Zypern',
 '             </option>',
 '            </select>',
 '            <svg class="icon_icon__ECGRl icon_appearance-default___4uy_" height="16px" viewbox="0 0 20 20" width="16px" xmlns="http://www.w3.org/2000/svg">',
 '             <path d="m0 5 10 10L20 5z" fill-rule="evenodd">',
 '             </path>',
 '            </svg>',
 '           </div>',
 '           <form class="styles_wrapper__bsj8o styles_locationField__3Q3cb">',
 '            <div class="styles_inputWrapper__8RN0V">',
 '             <div class="styles_inputIcon___dEI8 styles_searchIcon__BqP9U">',
 '              <svg class="icon_icon__ECGRl ic

### Identify the HTML elements containing the name, score, number of votes and site of each energy distributor.

On the page, the energy suppliers are represented in a similar way, it is therefore logical that the structure of the web code is the same.

The page is not set up as a table but it is a sequence of div containers, with class 'paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2'.

Important key words are:
 - p 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2' for the name of the energy supplier
 - p 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7' with 'TrustScore' for the rating score of the energy supplier and the respective number of votes; alternate: span 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_trustScore__8emxJ' for TrustScore only
 - span 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_metadataItem__Qn_Q2 styles_location__ILZb0' for supplier's business location; alternate: div 'styles_metadataRow__pgwwW'
 - div 'card_cardContent__sFUOe styles_categoriesLabelsWrapper__AsHrM styles_mobile__wIl_O' for energy supplier categories, explaining if the company also offers gas or eco-power
 - a class="link_internal__7XN06 link_wrapper__5ZJEx styles_linkWrapper__UWs5j" for hyperlink to comment page

Using the findAll method, we will retrieve the HTML code containing relevant elements in a variable named ener_imdb.

In [13]:
#Retrieve relevant HTML code of page 1 --> find_all rows
ener_tp = page1_soup.findAll("div", attrs={'class': 'paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2'})


In [14]:
#Retrieve relevant HTML code of page 1 --> find_all rows
ener2_tp = page2_soup.findAll("div", attrs={'class': 'paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2'})


In order to achieve and test access to the required information, we will start with the first listed energy supplier on page 1.

In [16]:
# get the first item from ener_imdb
ener1 = ener_tp[0]

# write content of first energy supplier on page 1 (Octopus Energy Germany)
ener1.text

'Am relevantestenOctopus Energy GermanyTrustScore 4,8|8.392 BewertungenMünchen,\xa0DeutschlandStromversorgungsunternehmen·Energieversorger·Energieanbieter·Gasversorgungsunternehmen·ÖkostromanbieterStromversorgungsunternehmen·Energieversorger·Energieanbieter·Gasversorgungsunternehmen·ÖkostromanbieterNeueste Bewertungen\xa0'

In [17]:
# recover the name
name = ener1.find('p', attrs={'class': 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2'}).text
print(name)

Octopus Energy Germany


In [18]:
# recover the score
score = ener1.find('span', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_trustScore__8emxJ'}).text
print(score)

TrustScore 4,8


In [19]:
# recover the score only
score = ener1.find('span', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_trustScore__8emxJ'}).get_text().strip('TrustScore ')
print(score)

4,8


In [20]:
# recover the score and the attributes
score = ener1.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).text
print(score)


TrustScore 4,8|8.392 Bewertungen


In [21]:
# strip explanations
score = ener1.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).get_text().strip(' Bewertungen').strip('TrustScore ')
print(score)

4,8|8.392


In [22]:
# get categories
categ = ener1.find('div', attrs={'class': 'card_cardContent__sFUOe styles_categoriesLabelsWrapper__AsHrM styles_mobile__wIl_O'}).get_text()
print(categ)

Stromversorgungsunternehmen·Energieversorger·Energieanbieter·Gasversorgungsunternehmen·Ökostromanbieter


In [23]:
# recover the location
location = ener1.find('div', attrs={'class': 'styles_metadataRow__pgwwW'}).get_text()
print(location)

München, Deutschland


In [24]:
# get the hyperlink for customers' comment page
comment = 'https://de.trustpilot.com' + ener1.find('a', attrs={'class': 'link_internal__7XN06 link_wrapper__5ZJEx styles_linkWrapper__UWs5j'})['href']
print(comment)

https://de.trustpilot.com/review/octopusenergy.de


### Collect the energy supplier data in a data frame

We can use a for loop to iterate on the ener_tp variable of the first page, containing all the pertinent information on the energy suppliers. 

The information will be stored in lists, which can then be transformed in a data frame with the zip function.

These steps can be repeated for the second page as well, stored into the ener2_tp variable.

Finally, both data frames can be concatentated to create one data frame.

In [26]:
## Page 1
#set variable lists for for-loop
names = []
scores = []
locations = []
categories = []
comments = []

#iterate through table rows with for loop
for supplier in ener_tp:

    #find name and append to list
    try:
        name = supplier.find('p', attrs={'class': 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2'}).text
    except Exception as e: 
        name = "Not Available"
    names.append(name)
    
    #find TrustScore and append to list
    try: 
        score = supplier.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).get_text().strip(' Bewertungen').strip('TrustScore ')
    except Exception as e: 
        score = "0,0|0"
    scores.append(score)
    
    #find location and append to list
    try:
        location = supplier.find('div', attrs={'class': 'styles_metadataRow__pgwwW'}).get_text()
    except Exception as e:
        location = "Not Available"
    locations.append(location)
    
    #find categories and append to list
    try:
        categ = supplier.find('div', attrs={'class': 'card_cardContent__sFUOe styles_categoriesLabelsWrapper__AsHrM styles_mobile__wIl_O'}).get_text()
    except Exception as e:
        categ = "Not Available"
    categories.append(categ)
    
    #find link to customer comment sit
    try:
        comment = 'https://de.trustpilot.com' + supplier.find('a', attrs={'class': 'link_internal__7XN06 link_wrapper__5ZJEx styles_linkWrapper__UWs5j'})['href']
    except Exception as e:
        comment = "Not Available"
    comments.append(comment)

#check with print
print("Names (first 5 from list): \n", names[0:5])
print("Scores (first 5 from list): \n", scores[0:5])
print("Locations (first 5 from list): \n", locations[0:5])
print("Categories (first 5 from list): \n", categories[0:5])
print("Comments (first 5 from list: \n", comments[0:5])

Names (first 5 from list): 
 ['Octopus Energy Germany', 'Ostrom', 'Rabot Charge', 'MONTANA Group', 'E.ON Energie Deutschland GmbH']
Scores (first 5 from list): 
 ['4,8|8.392', '4,8|1.607', '4,3|176', '4,0|3.153', '3,7|13.467']
Locations (first 5 from list): 
 ['München,\xa0Deutschland', 'Berlin,\xa0Deutschland', 'Hamburg,\xa0Deutschland', 'Grünwald,\xa0Deutschland', 'München,\xa0Deutschland']
Categories (first 5 from list): 
 ['Stromversorgungsunternehmen·Energieversorger·Energieanbieter·Gasversorgungsunternehmen·Ökostromanbieter', 'Ökostromanbieter·Stromversorgungsunternehmen·Energieanbieter·Solarenergieunternehmen·Energieversorger', 'Ökostromanbieter·Energieanbieter·Energieversorger·Stromversorgungsunternehmen', 'Kraftstofflieferant·Energieanbieter·Stromversorgungsunternehmen·Ökostromanbieter·Gasversorgungsunternehmen·Mineralölunternehmen', 'Solartechnikanbieter·Energieanbieter·Stromversorgungsunternehmen·Energieversorger·Gasversorgungsunternehmen·Ökostromanbieter']
Comments (first 5

In [27]:
#create dataframe with zip function
suppliers = pd.DataFrame(list(zip(names, locations, scores, categories, comments)), columns=["supplier", "location", "score_votes", "cat", "comment"])

#check
suppliers.head()


,supplier,location,score_votes,cat,comment
0,Octopus Energy Germany,"München, Deutschland","4,8|8.392",Stromversorgungsunternehmen·Energieversorger·E...,https://de.trustpilot.com/review/octopusenergy.de
1,Ostrom,"Berlin, Deutschland","4,8|1.607",Ökostromanbieter·Stromversorgungsunternehmen·E...,https://de.trustpilot.com/review/ostrom.de
2,Rabot Charge,"Hamburg, Deutschland","4,3|176",Ökostromanbieter·Energieanbieter·Energieversor...,https://de.trustpilot.com/review/rabot-charge.de
3,MONTANA Group,"Grünwald, Deutschland","4,0|3.153",Kraftstofflieferant·Energieanbieter·Stromverso...,https://de.trustpilot.com/review/montana-energ...
4,E.ON Energie Deutschland GmbH,"München, Deutschland","3,7|13.467",Solartechnikanbieter·Energieanbieter·Stromvers...,https://de.trustpilot.com/review/eon.de


In [28]:
suppliers.info()
# 20 entries is correct, these are displayed on first result page

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   supplier     20 non-null     object
 1   location     20 non-null     object
 2   score_votes  20 non-null     object
 3   cat          20 non-null     object
 4   comment      20 non-null     object
dtypes: object(5)
memory usage: 932.0+ bytes


In [29]:
## Page 2
#set variable lists for for-loop
names = []
scores = []
locations = []
categories = []
comments = []

#iterate through table rows with for loop
for supplier in ener2_tp:

    #find name and append to list
    try:
        name = supplier.find('p', attrs={'class': 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2'}).text
    except Exception as e: 
        name = "Not Available"
    names.append(name)
    
    #find TrustScore and append to list
    try: 
        score = supplier.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).get_text().strip(' Bewertungen').strip('TrustScore ')
    except Exception as e: 
        score = "0,0|0"
    scores.append(score)
    
    #find location and append to list
    try:
        location = supplier.find('div', attrs={'class': 'styles_metadataRow__pgwwW'}).get_text()
    except Exception as e:
        location = "Not Available"
    locations.append(location)
    
    #find categories and append to list
    try:
        categ = supplier.find('div', attrs={'class': 'card_cardContent__sFUOe styles_categoriesLabelsWrapper__AsHrM styles_mobile__wIl_O'}).get_text()
    except Exceptions as e:
        categ = "Not Available"
    categories.append(categ)
    
     #find link to customer comment sit
    try:
        comment = 'https://de.trustpilot.com' + supplier.find('a', attrs={'class': 'link_internal__7XN06 link_wrapper__5ZJEx styles_linkWrapper__UWs5j'})['href']
    except Exception as e:
        comment = "Not Available"
    comments.append(comment)

#check with print
print("Names (first 5 from list): \n", names[0:5])
print("Scores (first 5 from list): \n", scores[0:5])
print("Locations (first 5 from list): \n", locations[0:5])
print("Categories (first 5 from list): \n", categories[0:5])
print("Comments (first 5 from list: \n", comments[0:5])


Names (first 5 from list): 
 ['Gas-und-Stromvergleich.de', 'nowenergy', 'CHONO MEDIA', 'Fresh Energy', 'Polarstern Energie']
Scores (first 5 from list): 
 ['0,0|0', '3,4|81', '3,2|39', '3,1|68', '3,1|32']
Locations (first 5 from list): 
 ['', 'Berlin,\xa0Deutschland', '', 'Berlin,\xa0Deutschland', 'München,\xa0Deutschland']
Categories (first 5 from list): 
 ['Stromversorgungsunternehmen·Gasversorgungsunternehmen', 'Gasversorgungsunternehmen·Stromversorgungsunternehmen·Internetanbieter·Energieversorger·Telekommunikationsanbieter·Telefon- und Internetdienst', 'Reiseanbieter·Telekommunikationsanbieter·Stromversorgungsunternehmen·Online-Marktplatz', 'Stromversorgungsunternehmen', 'Energieanbieter·Ökostromanbieter·Stromversorgungsunternehmen·Gasversorgungsunternehmen']
Comments (first 5 from list: 
 ['https://de.trustpilot.com/review/www.gas-und-stromvergleich.de', 'https://de.trustpilot.com/review/www.nowenergy.de', 'https://de.trustpilot.com/review/www.chono.de', 'https://de.trustpilot.co

In [30]:
#create dataframe with zip function
suppliers2 = pd.DataFrame(list(zip(names, locations, scores, categories, comments)), columns=["supplier", "location", "score_votes", "cat", "comment"])

#check
suppliers2.head()


,supplier,location,score_votes,cat,comment
0,Gas-und-Stromvergleich.de,,"0,0|0",Stromversorgungsunternehmen·Gasversorgungsunte...,https://de.trustpilot.com/review/www.gas-und-s...
1,nowenergy,"Berlin, Deutschland","3,4|81",Gasversorgungsunternehmen·Stromversorgungsunte...,https://de.trustpilot.com/review/www.nowenergy.de
2,CHONO MEDIA,,"3,2|39",Reiseanbieter·Telekommunikationsanbieter·Strom...,https://de.trustpilot.com/review/www.chono.de
3,Fresh Energy,"Berlin, Deutschland","3,1|68",Stromversorgungsunternehmen,https://de.trustpilot.com/review/getfresh.energy
4,Polarstern Energie,"München, Deutschland","3,1|32",Energieanbieter·Ökostromanbieter·Stromversorgu...,https://de.trustpilot.com/review/polarstern-en...


In [31]:
suppliers2.info()
# 17 entries is correct, these are displayed on second result page

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   supplier     17 non-null     object
 1   location     17 non-null     object
 2   score_votes  17 non-null     object
 3   cat          17 non-null     object
 4   comment      17 non-null     object
dtypes: object(5)
memory usage: 812.0+ bytes


In [32]:
## Use pandas.concat() method to unify the supplier data frames
ener_suppliers = pd.concat([suppliers, suppliers2], ignore_index=True, sort=False)

In [33]:
# check
display(ener_suppliers)

,supplier,location,score_votes,cat,comment
0,Octopus Energy Germany,"München, Deutschland","4,8|8.392",Stromversorgungsunternehmen·Energieversorger·E...,https://de.trustpilot.com/review/octopusenergy.de
1,Ostrom,"Berlin, Deutschland","4,8|1.607",Ökostromanbieter·Stromversorgungsunternehmen·E...,https://de.trustpilot.com/review/ostrom.de
2,Rabot Charge,"Hamburg, Deutschland","4,3|176",Ökostromanbieter·Energieanbieter·Energieversor...,https://de.trustpilot.com/review/rabot-charge.de
3,MONTANA Group,"Grünwald, Deutschland","4,0|3.153",Kraftstofflieferant·Energieanbieter·Stromverso...,https://de.trustpilot.com/review/montana-energ...
4,E.ON Energie Deutschland GmbH,"München, Deutschland","3,7|13.467",Solartechnikanbieter·Energieanbieter·Stromvers...,https://de.trustpilot.com/review/eon.de
5,Grünwelt Energie,"Kaarst, Deutschland","3,6|1.966",Stromversorgungsunternehmen,https://de.trustpilot.com/review/www.gruenwelt.de
6,RheinEnergie,"Köln, Deutschland","3,4|547",Stromversorgungsunternehmen·Energieanbieter·Ök...,https://de.trustpilot.com/review/rheinenergie.com
7,badenova,"Freiburg im Breisgau, Deutschland","2,8|242",Stromversorgungsunternehmen·Ökostromanbieter·G...,https://de.trustpilot.com/review/www.badenova.de
8,pricewise.de,"Heidelberg, Deutschland","4,8|119",Gasversorgungsunternehmen·Stromversorgungsunte...,https://de.trustpilot.com/review/www.prizewize.de
9,DFM-Select GmbH,"Metzingen, Deutschland","4,6|22",Anbieter von Elektronikbauteilen·Technischer K...,https://de.trustpilot.com/review/dfm-select.de


In [34]:
# Saving the new data frame before going on with the next steps
ener_suppliers.to_csv("ener_suppliers_raw.csv", index=False)

### Define function to automate process

In [36]:
import requests
from bs4 import BeautifulSoup
import csv
import re

base_url = "https://de.trustpilot.com/categories/electric_utility_company"

def scrap_ener(): 
    csv_file = open("fliplart-data.csv", "a")
    writer = csv.writer(csv_file)

    writer.writerow(
        ['Product_name', 'Price', 'Rating', 'Product-url'])

    for page in range(1, 510):

        page = base_url + str(page)

        response = requests.get(page).text

        soup = BeautifulSoup(response, 'lxml')

        for product_urls in soup.find_all('a', href=True, attrs={'class': '_1fQZEK'}):
            
            #name
            try:
                name = product_urls.find('div', attrs={'class': '_4rR01T'}).text
            except Exception as e:
                name = "Not Available"

            #price
            try:
                price = product_urls.find('div', attrs={'class': '_30jeq3 _1_WHN1'}).text
                price = re.split("\₹", price)
                price = price[-1]
            except Exception as e:
                price = "Not Available"

            #rating
            try:
                rating = product_urls.find('div', attrs={'class': '_3LWZlK'}).text
            except Exception as e:
                rating = "Not Available"
            #item_url
            try:
                item_url = soup.find('a', class_="_1fQZEK", target="_blank")['href']
                item_url = " https://www.flipkart.com" + item_url
                item_url = re.split("\&", item_url)
                item_url = item_url[0]
            except Exception as e:
                item_url = "Not Available"

            print(f'Product name is {name}')
            print(f'Product price is {price}')
            print(f'Product rating is {rating}')
            print(f'Product url is {item_url}')


            writer.writerow(
                [name, price, rating, item_url])


In [37]:
import requests
from bs4 import BeautifulSoup
import csv
import re

base_url = "https://de.trustpilot.com/categories/electric_utility_company"

def get_urls(): 
    csv_file = open("fliplart-data.csv", "a")
    writer = csv.writer(csv_file)

    writer.writerow(
        ['Product_name', 'Price', 'Rating', 'Product-url'])

    for page in range(1, 510):

        page = base_url + str(page)

        response = requests.get(page).text

        soup = BeautifulSoup(response, 'lxml')

        for product_urls in soup.find_all('a', href=True, attrs={'class': '_1fQZEK'}):
            
            #name
            try:
                name = product_urls.find('div', attrs={'class': '_4rR01T'}).text
            except Exception as e:
                name = "Not Available"

            #price
            try:
                price = product_urls.find('div', attrs={'class': '_30jeq3 _1_WHN1'}).text
                price = re.split("\₹", price)
                price = price[-1]
            except Exception as e:
                price = "Not Available"

            #rating
            try:
                rating = product_urls.find('div', attrs={'class': '_3LWZlK'}).text
            except Exception as e:
                rating = "Not Available"
            #item_url
            try:
                item_url = soup.find('a', class_="_1fQZEK", target="_blank")['href']
                item_url = " https://www.flipkart.com" + item_url
                item_url = re.split("\&", item_url)
                item_url = item_url[0]
            except Exception as e:
                item_url = "Not Available"

            print(f'Product name is {name}')
            print(f'Product price is {price}')
            print(f'Product rating is {rating}')
            print(f'Product url is {item_url}')


            writer.writerow(
                [name, price, rating, item_url])


### Clean data frame

In [39]:
df = ener_suppliers 

In [40]:
df.head()

,supplier,location,score_votes,cat,comment
0,Octopus Energy Germany,"München, Deutschland","4,8|8.392",Stromversorgungsunternehmen·Energieversorger·E...,https://de.trustpilot.com/review/octopusenergy.de
1,Ostrom,"Berlin, Deutschland","4,8|1.607",Ökostromanbieter·Stromversorgungsunternehmen·E...,https://de.trustpilot.com/review/ostrom.de
2,Rabot Charge,"Hamburg, Deutschland","4,3|176",Ökostromanbieter·Energieanbieter·Energieversor...,https://de.trustpilot.com/review/rabot-charge.de
3,MONTANA Group,"Grünwald, Deutschland","4,0|3.153",Kraftstofflieferant·Energieanbieter·Stromverso...,https://de.trustpilot.com/review/montana-energ...
4,E.ON Energie Deutschland GmbH,"München, Deutschland","3,7|13.467",Solartechnikanbieter·Energieanbieter·Stromvers...,https://de.trustpilot.com/review/eon.de


In [41]:
## separate scores and votes in 2 separate columns score and votes
df[['score', 'votes']] = df['score_votes'].apply(lambda x: pd.Series(str(x).split("|")))

# remove column score_votes
df = df.drop(['score_votes'], axis= 1)

#check
df.head()

,supplier,location,cat,comment,score,votes
0,Octopus Energy Germany,"München, Deutschland",Stromversorgungsunternehmen·Energieversorger·E...,https://de.trustpilot.com/review/octopusenergy.de,"4,8",8.392
1,Ostrom,"Berlin, Deutschland",Ökostromanbieter·Stromversorgungsunternehmen·E...,https://de.trustpilot.com/review/ostrom.de,"4,8",1.607
2,Rabot Charge,"Hamburg, Deutschland",Ökostromanbieter·Energieanbieter·Energieversor...,https://de.trustpilot.com/review/rabot-charge.de,"4,3",176
3,MONTANA Group,"Grünwald, Deutschland",Kraftstofflieferant·Energieanbieter·Stromverso...,https://de.trustpilot.com/review/montana-energ...,"4,0",3.153
4,E.ON Energie Deutschland GmbH,"München, Deutschland",Solartechnikanbieter·Energieanbieter·Stromvers...,https://de.trustpilot.com/review/eon.de,"3,7",13.467


In [42]:
## Alternate solution: separate scores and votes in 2 separate columns score and votes
# copy column
#df['votes'] = df['score_votes']

# remove first 4 strings to receive votes
#df['votes'] = df['votes'].str[4:]

# keep first 4 strings to keep score
#df['score_votes'] = df['score_votes'].str[:3]

# rename column
#df = df.rename(columns={'score_votes': 'score'})

#check
#df.head()

In [43]:
## change datatypes to float for score and change decimal separator to '.'
df['score'] = df['score'].str.replace(",",".")
df['score'] = df['score'].astype('float')
df.tail()

,supplier,location,cat,comment,score,votes
32,RWE,"Essen, Deutschland",Stromversorgungsunternehmen,https://de.trustpilot.com/review/www.rwe.de,1.3,90
33,EWE,,Stromversorgungsunternehmen,https://de.trustpilot.com/review/www.ewe.de,1.2,2.239
34,Mein Stromgas,"Dorsten, Deutschland",Stromversorgungsunternehmen,https://de.trustpilot.com/review/mein-stromgas.de,0.0,0
35,OPTUM GmbH,,Gasversorgungsunternehmen·Ökostromanbieter·Str...,https://de.trustpilot.com/review/optum-eba.de,0.0,0
36,Netzberatung,"Kassel, Deutschland",Energieanbieter·Stromversorgungsunternehmen·Ve...,https://de.trustpilot.com/review/netzberatung....,0.0,0


In [44]:
## remove '.' from votes and change data type to integer
df['votes'] = df['votes'].str.replace(".","")
df['votes'] = df['votes'].astype('int64')
df.head()

/tmp/ipykernel_328/539288907.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['votes'] = df['votes'].str.replace(".","")


,supplier,location,cat,comment,score,votes
0,Octopus Energy Germany,"München, Deutschland",Stromversorgungsunternehmen·Energieversorger·E...,https://de.trustpilot.com/review/octopusenergy.de,4.8,8392
1,Ostrom,"Berlin, Deutschland",Ökostromanbieter·Stromversorgungsunternehmen·E...,https://de.trustpilot.com/review/ostrom.de,4.8,1607
2,Rabot Charge,"Hamburg, Deutschland",Ökostromanbieter·Energieanbieter·Energieversor...,https://de.trustpilot.com/review/rabot-charge.de,4.3,176
3,MONTANA Group,"Grünwald, Deutschland",Kraftstofflieferant·Energieanbieter·Stromverso...,https://de.trustpilot.com/review/montana-energ...,4.0,3153
4,E.ON Energie Deutschland GmbH,"München, Deutschland",Solartechnikanbieter·Energieanbieter·Stromvers...,https://de.trustpilot.com/review/eon.de,3.7,13467


In [45]:
## Split location into two columns use apply()
df[['city', 'country']] = df['location'].apply(lambda x: pd.Series(str(x).split(",")))

# replace " " before country name
df['country'] = df['country'].str.replace(" ","")

# remove column location
df = df.drop(['location'], axis= 1)

#check
df.head()


,supplier,cat,comment,score,votes,city,country
0,Octopus Energy Germany,Stromversorgungsunternehmen·Energieversorger·E...,https://de.trustpilot.com/review/octopusenergy.de,4.8,8392,München,Deutschland
1,Ostrom,Ökostromanbieter·Stromversorgungsunternehmen·E...,https://de.trustpilot.com/review/ostrom.de,4.8,1607,Berlin,Deutschland
2,Rabot Charge,Ökostromanbieter·Energieanbieter·Energieversor...,https://de.trustpilot.com/review/rabot-charge.de,4.3,176,Hamburg,Deutschland
3,MONTANA Group,Kraftstofflieferant·Energieanbieter·Stromverso...,https://de.trustpilot.com/review/montana-energ...,4.0,3153,Grünwald,Deutschland
4,E.ON Energie Deutschland GmbH,Solartechnikanbieter·Energieanbieter·Stromvers...,https://de.trustpilot.com/review/eon.de,3.7,13467,München,Deutschland


In [46]:
# replace "·" before in column cat with " "
df['cat'] = df['cat'].str.replace("·"," ")


In [47]:
# check
df.head()

,supplier,cat,comment,score,votes,city,country
0,Octopus Energy Germany,Stromversorgungsunternehmen Energieversorger E...,https://de.trustpilot.com/review/octopusenergy.de,4.8,8392,München,Deutschland
1,Ostrom,Ökostromanbieter Stromversorgungsunternehmen E...,https://de.trustpilot.com/review/ostrom.de,4.8,1607,Berlin,Deutschland
2,Rabot Charge,Ökostromanbieter Energieanbieter Energieversor...,https://de.trustpilot.com/review/rabot-charge.de,4.3,176,Hamburg,Deutschland
3,MONTANA Group,Kraftstofflieferant Energieanbieter Stromverso...,https://de.trustpilot.com/review/montana-energ...,4.0,3153,Grünwald,Deutschland
4,E.ON Energie Deutschland GmbH,Solartechnikanbieter Energieanbieter Stromvers...,https://de.trustpilot.com/review/eon.de,3.7,13467,München,Deutschland


In [48]:
##  rearrange order of columns
# get columns list
cols = df.columns.tolist()
cols

['supplier', 'cat', 'comment', 'score', 'votes', 'city', 'country']

In [49]:
#create new list with updated order
cols_new = ['supplier', 'city', 'country', 'cat', 'score', 'votes', 'comment']

#rearrange columns
df = df[cols_new]

#check
df.head()

,supplier,city,country,cat,score,votes,comment
0,Octopus Energy Germany,München,Deutschland,Stromversorgungsunternehmen Energieversorger E...,4.8,8392,https://de.trustpilot.com/review/octopusenergy.de
1,Ostrom,Berlin,Deutschland,Ökostromanbieter Stromversorgungsunternehmen E...,4.8,1607,https://de.trustpilot.com/review/ostrom.de
2,Rabot Charge,Hamburg,Deutschland,Ökostromanbieter Energieanbieter Energieversor...,4.3,176,https://de.trustpilot.com/review/rabot-charge.de
3,MONTANA Group,Grünwald,Deutschland,Kraftstofflieferant Energieanbieter Stromverso...,4.0,3153,https://de.trustpilot.com/review/montana-energ...
4,E.ON Energie Deutschland GmbH,München,Deutschland,Solartechnikanbieter Energieanbieter Stromvers...,3.7,13467,https://de.trustpilot.com/review/eon.de


In [50]:
#check transformation with info
df.info()
# no missing values, dtypes correct

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   supplier  37 non-null     object 
 1   city      37 non-null     object 
 2   country   31 non-null     object 
 3   cat       37 non-null     object 
 4   score     37 non-null     float64
 5   votes     37 non-null     int64  
 6   comment   37 non-null     object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.2+ KB


In [51]:
df.describe()

,score,votes
count,37.000000,37.000000
mean,2.964865,1505.243243
std,1.457779,3240.455218
min,0.000000,0.000000
25%,1.900000,39.000000
50%,3.400000,125.000000
75%,4.000000,961.000000
max,4.800000,13467.000000


In [101]:
display(df)

,supplier,city,country,cat,score,votes,comment
0,Octopus Energy Germany,München,Deutschland,Stromversorgungsunternehmen Energieversorger E...,4.8,8392,https://de.trustpilot.com/review/octopusenergy.de
1,Ostrom,Berlin,Deutschland,Ökostromanbieter Stromversorgungsunternehmen E...,4.8,1607,https://de.trustpilot.com/review/ostrom.de
2,Rabot Charge,Hamburg,Deutschland,Ökostromanbieter Energieanbieter Energieversor...,4.3,176,https://de.trustpilot.com/review/rabot-charge.de
3,MONTANA Group,Grünwald,Deutschland,Kraftstofflieferant Energieanbieter Stromverso...,4.0,3153,https://de.trustpilot.com/review/montana-energ...
4,E.ON Energie Deutschland GmbH,München,Deutschland,Solartechnikanbieter Energieanbieter Stromvers...,3.7,13467,https://de.trustpilot.com/review/eon.de
5,Grünwelt Energie,Kaarst,Deutschland,Stromversorgungsunternehmen,3.6,1966,https://de.trustpilot.com/review/www.gruenwelt.de
6,RheinEnergie,Köln,Deutschland,Stromversorgungsunternehmen Energieanbieter Ök...,3.4,547,https://de.trustpilot.com/review/rheinenergie.com
7,badenova,Freiburg im Breisgau,Deutschland,Stromversorgungsunternehmen Ökostromanbieter G...,2.8,242,https://de.trustpilot.com/review/www.badenova.de
8,pricewise.de,Heidelberg,Deutschland,Gasversorgungsunternehmen Stromversorgungsunte...,4.8,119,https://de.trustpilot.com/review/www.prizewize.de
9,DFM-Select GmbH,Metzingen,Deutschland,Anbieter von Elektronikbauteilen Technischer K...,4.6,22,https://de.trustpilot.com/review/dfm-select.de


### Final CSV of energy supplier ratings Germany for export

In [53]:
# Saving the cleaned data frame in a csv file
df.to_csv("ener_supplier_rankings_clean.csv", index=False)